# Flaw Engine: Cloud Trainer (Gideon-gg101 Edition) 🚀

This notebook turns this Colab instance into a **Distributed Worker** for the Flaw Chess Engine. It plays games against itself and sends the tactical findings back to your local Master Engine.

### 🏁 Setup Options:
Choose **either** Section 1.1 (GitHub) **or** Section 1.2 (Manual ZIP).

## 1.1 Option A: Setup via GitHub (Recommended)
Connect your GitHub repository to pull the latest code automatically.

In [ ]:
GITHUB_URL = "https://github.com/Gideon-gg101/flaw-engine.git"
!git clone {GITHUB_URL} flaw
%cd /content/flaw

## 1.2 Option B: Setup via Manual ZIP
Upload your `flaw_cloud_bundle.zip` to the file explorer on the left, then run this.

In [ ]:
!unzip -o /content/flaw_cloud_bundle.zip -d /content/flaw
%cd /content/flaw

## 2. Prepare Environment & Build C++ Core

In [ ]:
!apt-get install -y cmake g++
!pip install requests torch

# Ensure we are in the root
%cd /content/flaw

!mkdir -p core/build
%cd core/build
!cmake ..
!make -j$(nproc)
# Move the built library to the root so Python can find it
!cp flaw_core*.so ../../

# Return to root
%cd /content/flaw

## 3. Start Live Training (Sync to Local PC)
Replace the `MASTER_URL` with your local `ngrok` URL.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"

# Robust launch: ensure we are in the right folder and set PYTHONPATH
!export PYTHONPATH=$PYTHONPATH:/content/flaw && cd /content/flaw && python3 -m ai.selfplay_worker --master {MASTER_URL}